In [2]:
import pandas as pd
df = pd.read_csv('/Users/mahmoud/Zillow_Time_Series/zillow_dataaa.csv')

In [3]:
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


In [5]:
df= df.drop(columns=['RegionID'])

In [6]:
df.head()

,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,337600.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,235400.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,210700.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,504600.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [7]:
df.shape

(14723, 271)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Preprocess the data
# Handle missing values or outliers, perform any necessary transformations or feature engineering

# Convert to long format
df_long = df.melt(id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='Date', value_name='States')
df_long['Date'] = pd.to_datetime(df_long['Date'])

# Split the data into training and testing sets
n_test = 12  # Number of test periods
train_data = df_long.iloc[:-n_test]
test_data = df_long.iloc[-n_test:]

# Define SARIMA model parameters
p = 1  # Autoregressive order
d = 1  # Integration order
q = 1  # Moving average order
P = 1  # Seasonal autoregressive order
D = 1  # Seasonal integration order
Q = 1  # Seasonal moving average order
m = 12  # Seasonal period

# Fit the SARIMA model
model = SARIMAX(train_data['State'], order=(p, d, q), seasonal_order=(P, D, Q, m))
model_fit = model.fit()

# Validate the model
predictions = model_fit.predict(start=test_data.index[0], end=test_data.index[-1])

# Evaluate the model
rmse = np.sqrt(((predictions - test_data['State']) ** 2).mean())
print("Root Mean Squared Error (RMSE):", rmse)

# Forecast future values
future_periods = 12  # number of periods to forecast
forecast = model_fit.get_forecast(steps=future_periods)
forecasted_values = forecast.predicted_mean

# Evaluate the forecast
predictions = model_fit.get_forecast(steps=len(test_data))
predicted_values = predictions.predicted_mean

# Compare forecasted_values with actual values of 'States'
comparison_df = pd.DataFrame({'Actual': test_data['State'], 'Predicted': predicted_values})

# Visualize the forecast and actual values
plt.plot(test_data['Date'], test_data['State'], label='Actual')
plt.plot(test_data['Date'], predicted_values, label='Predicted')
plt.xlabel('Date')
plt.ylabel('State')
plt.legend()
plt.show()

# Plot the time series of 'States' for each time period and overlay the forecasted values
mse = mean_squared_error(test_data['States'], predicted_values)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")